In [1]:
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

/Users/m-suzuki/Projects/bert-japanese/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-19 10:03:53.353302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_name_or_path = "cl-tohoku/bert-base-japanese-v3"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [4]:
model = AutoModelForMaskedLM.from_pretrained(model_name_or_path)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v3 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
input_ids = tokenizer.encode(f"青葉山で{tokenizer.mask_token}の研究をしています。", return_tensors="pt")

In [6]:
print(input_ids)

tensor([[    2, 22033,  1872,   457,     4,   464, 12605,   500,   441,   456,
           422, 12995,   385,     3]])


In [7]:
print(tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))

['[CLS]', '青葉', '山', 'で', '[MASK]', 'の', '研究', 'を', 'し', 'て', 'い', 'ます', '。', '[SEP]']


In [8]:
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].tolist()
print(masked_index)

4


In [9]:
result = model(input_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))


[CLS] 青葉 山 で 雪 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 山 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 花 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 植物 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 星 の 研究 を し て い ます 。 [SEP]
